In [29]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib import cm as cm
import numpy as np
import re


from pymystem3 import Mystem

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Download preprocessed df

In [30]:
df = pd.read_csv('./preprocessed_df.csv')
df.shape
df.head()

,comment,date_time,color,size,thumb_up,thumb_down,prod_eval,prod,brand,cleaned_comment,lemma_comment,lemma_comment_2,words_count
0,"Хорошее колечко, получила 20 марта, ни один ка...",2021-04-20T09:13:31Z,серебристый,18,0,1,4,Кольцо,SOKOLOV,хорошее колечко получила марта камушек выпал н...,колечко получать марта камушек выпадать носить...,получать марта камушек выпадать носить либо по...,12
1,Спустя месяц выпал камень при аккуратном ношен...,2021-03-09T20:34:29Z,серебристый,17,1,0,1,Кольцо,SOKOLOV,спустя месяц выпал камень аккуратном ношении и...,спустя месяц выпадать камень аккуратный ношени...,спустя месяц выпадать аккуратный ношение интер...,11
2,"Очееень красивое колечко, очень жаль, что выпа...",2021-02-02T09:54:05Z,серебристый,17,1,1,5,Кольцо,SOKOLOV,очееень красивое колечко жаль выпал стразик пр...,очееень красивый колечко жаль выпадать стразик...,очееень жаль выпадать стразик проходить месяц,6
3,"Очень красивое, изящное и нежное колечко, каму...",2021-01-29T18:14:08Z,серебристый,16,1,0,5,Кольцо,SOKOLOV,красивое изящное нежное колечко камушки сверкают,красивый изящный нежный колечко камушек сверкать,изящный нежный камушек сверкать,4
4,"Кольцо само по себе ну оч красивое, и камушки ...",2021-01-23T10:40:45Z,серебристый,16,3,2,3,Кольцо,SOKOLOV,красивое камушки блестят размер ойй знаю проко...,красивый камушек блестеть размер ойй прокол ра...,камушек блестеть ойй прокол оооочень еле мизин...,10


# Clustering

In [31]:
df.head(3)

,comment,date_time,color,size,thumb_up,thumb_down,prod_eval,prod,brand,cleaned_comment,lemma_comment,lemma_comment_2,words_count
0,"Хорошее колечко, получила 20 марта, ни один ка...",2021-04-20T09:13:31Z,серебристый,18,0,1,4,Кольцо,SOKOLOV,хорошее колечко получила марта камушек выпал н...,колечко получать марта камушек выпадать носить...,получать марта камушек выпадать носить либо по...,12
1,Спустя месяц выпал камень при аккуратном ношен...,2021-03-09T20:34:29Z,серебристый,17,1,0,1,Кольцо,SOKOLOV,спустя месяц выпал камень аккуратном ношении и...,спустя месяц выпадать камень аккуратный ношени...,спустя месяц выпадать аккуратный ношение интер...,11
2,"Очееень красивое колечко, очень жаль, что выпа...",2021-02-02T09:54:05Z,серебристый,17,1,1,5,Кольцо,SOKOLOV,очееень красивое колечко жаль выпал стразик пр...,очееень красивый колечко жаль выпадать стразик...,очееень жаль выпадать стразик проходить месяц,6


In [32]:
comments_clean = df['lemma_comment_2']

In [33]:
#Matrix document - vocabularly, vocab - 1000 words
tfidf_clust = TfidfVectorizer(1000).fit(comments_clean)
X_matrix =  tfidf_clust.transform(comments_clean).toarray()
X_matrix.shape

/opt/homebrew/Caskroom/miniforge/base/envs/tf_env/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass input=1000 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


(91, 533)

In [34]:
#fitting clust model
clf = KMeans(n_clusters=5, max_iter=1000, n_init=5, random_state = 10)
clf.fit(X_matrix)
cluster_labels = clf.labels_
        
clust_centers = clf.cluster_centers_
        
silhouette = silhouette_score(X_matrix, cluster_labels, metric='euclidean')


In [35]:
clust_centers

array([[ 7.36998160e-03,  1.24491289e-02, -4.33680869e-19, ...,
         4.19578790e-03,  1.63639139e-02,  1.73472348e-18],
       [-1.30104261e-18,  0.00000000e+00,  0.00000000e+00, ...,
         1.75649509e-02,  8.67361738e-19,  0.00000000e+00],
       [-1.30104261e-18,  3.75975248e-02,  1.73464397e-02, ...,
         8.67361738e-19,  0.00000000e+00,  3.23598996e-02],
       [-1.30104261e-18,  1.73472348e-18,  0.00000000e+00, ...,
         8.67361738e-19,  0.00000000e+00,  8.67361738e-19],
       [-1.30104261e-18,  1.73472348e-18, -4.33680869e-19, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [36]:
silhouette

0.00975384531473908

In [37]:
unique, counts = np.unique(cluster_labels, return_counts=True)
print (np.asarray((unique, counts)).T)

[[ 0 40]
 [ 1 15]
 [ 2 13]
 [ 3 13]
 [ 4 10]]


In [38]:
X_matrix.shape, cluster_labels.shape, comments_clean.shape

((91, 533), (91,), (91,))

In [39]:
df_clust = pd.DataFrame(df['comment'].to_numpy(), columns = ['comment'])

In [40]:
#add cluster label to each comment
df_clust['cluster'] = cluster_labels

In [41]:
df_clust['lemma_comment_2'] = df['lemma_comment_2'] 

In [42]:
df_clust = df_clust.reset_index()

In [43]:
df_clust.head()

,index,comment,cluster,lemma_comment_2
0,0,"Хорошее колечко, получила 20 марта, ни один ка...",1,получать марта камушек выпадать носить либо по...
1,1,Спустя месяц выпал камень при аккуратном ношен...,0,спустя месяц выпадать аккуратный ношение интер...
2,2,"Очееень красивое колечко, очень жаль, что выпа...",0,очееень жаль выпадать стразик проходить месяц
3,3,"Очень красивое, изящное и нежное колечко, каму...",1,изящный нежный камушек сверкать
4,4,"Кольцо само по себе ну оч красивое, и камушки ...",2,камушек блестеть ойй прокол оооочень еле мизин...


In [44]:
df_clust['vector'] = df_clust['index'].apply(lambda x: X_matrix[x])

In [45]:
#sum all vector coordinas in order to delete 0 vectors
df_clust['sum_vec'] = df_clust['vector'].apply(lambda x: sum(x))

In [46]:
df_clust['vector']

0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1     [0.0, 0.0, 0.0, 0.23655001273557275, 0.0, 0.0,...
2     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                            ...                        
86    [0.0, 0.0, 0.0, 0.38779166648156, 0.0, 0.0, 0....
87    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
88    [0.0, 0.0, 0.0, 0.322199146604588, 0.0, 0.0, 0...
89    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
90    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: vector, Length: 91, dtype: object

In [47]:
df_clust = df_clust.drop(df_clust[df_clust['sum_vec'] == 0].index).reset_index(drop = True)

In [48]:
df_clust = df_clust.drop(columns='index').reset_index()

In [49]:
df_clust.head()

,index,comment,cluster,lemma_comment_2,vector,sum_vec
0,0,"Хорошее колечко, получила 20 марта, ни один ка...",1,получать марта камушек выпадать носить либо по...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.424584
1,1,Спустя месяц выпал камень при аккуратном ношен...,0,спустя месяц выпадать аккуратный ношение интер...,"[0.0, 0.0, 0.0, 0.23655001273557275, 0.0, 0.0,...",3.278803
2,2,"Очееень красивое колечко, очень жаль, что выпа...",0,очееень жаль выпадать стразик проходить месяц,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.424191
3,3,"Очень красивое, изящное и нежное колечко, каму...",1,изящный нежный камушек сверкать,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.978025
4,4,"Кольцо само по себе ну оч красивое, и камушки ...",2,камушек блестеть ойй прокол оооочень еле мизин...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.131941


# Calculation of cluster centers and distance to each comment

In [50]:
#calculation clust centers coordinats
df_clust['centr'] = df_clust['cluster'].apply(lambda x: clust_centers[x])
df_clust.head(3)

,index,comment,cluster,lemma_comment_2,vector,sum_vec,centr
0,0,"Хорошее колечко, получила 20 марта, ни один ка...",1,получать марта камушек выпадать носить либо по...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.424584,"[-1.3010426069826053e-18, 0.0, 0.0, 0.01485564..."
1,1,Спустя месяц выпал камень при аккуратном ношен...,0,спустя месяц выпадать аккуратный ношение интер...,"[0.0, 0.0, 0.0, 0.23655001273557275, 0.0, 0.0,...",3.278803,"[0.007369981596944172, 0.012449128860977711, -..."
2,2,"Очееень красивое колечко, очень жаль, что выпа...",0,очееень жаль выпадать стразик проходить месяц,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.424191,"[0.007369981596944172, 0.012449128860977711, -..."


In [51]:
#calculation distance from clust to centers to each comment
from scipy.spatial import distance
df_clust['dist'] = df_clust['index'].apply(lambda x: distance.euclidean(df_clust['centr'][x], df_clust['vector'][x]))
df_clust.head()

,index,comment,cluster,lemma_comment_2,vector,sum_vec,centr,dist
0,0,"Хорошее колечко, получила 20 марта, ни один ка...",1,получать марта камушек выпадать носить либо по...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.424584,"[-1.3010426069826053e-18, 0.0, 0.0, 0.01485564...",0.944140
1,1,Спустя месяц выпал камень при аккуратном ношен...,0,спустя месяц выпадать аккуратный ношение интер...,"[0.0, 0.0, 0.0, 0.23655001273557275, 0.0, 0.0,...",3.278803,"[0.007369981596944172, 0.012449128860977711, -...",0.966872
2,2,"Очееень красивое колечко, очень жаль, что выпа...",0,очееень жаль выпадать стразик проходить месяц,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.424191,"[0.007369981596944172, 0.012449128860977711, -...",0.972166
3,3,"Очень красивое, изящное и нежное колечко, каму...",1,изящный нежный камушек сверкать,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.978025,"[-1.3010426069826053e-18, 0.0, 0.0, 0.01485564...",0.900827
4,4,"Кольцо само по себе ну оч красивое, и камушки ...",2,камушек блестеть ойй прокол оооочень еле мизин...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.131941,"[-1.3010426069826053e-18, 0.03759752483382011,...",0.935140


# Functions (defining part of speech, primary list of topics and final topic  for each cluster)

In [52]:
#func which define part of speech (pos)
def pos_define(x):
    m = Mystem()
    w = m.analyze(x)[0]
    pos = w['analysis'][0]['gr'].split(',')[0]
    pos = pos.split('=')[0].strip()
    return pos

In [53]:
#final topic list forming
def final_topics(df):
    final_topics = []
    for x in df.index:
        if len(final_topics)<5:
            if pos_define(x) not in ['V','ADV'] and (x not in final_topics):
                final_topics.append(x)
        else: break
    return final_topics

In [54]:
#primary topic list defining
def clust_topics_define(df):
    clust_num = df['cluster'].nunique()
    
    clust_topics = {}
    
    for i in range(clust_num):
        topl = []
        vectorizer = TfidfVectorizer(min_df=2)
        df_filt = df[df['cluster'] == i]['lemma_comment_2']
        vectorizer.fit_transform(df_filt)
        
        idf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
        idf = pd.DataFrame(columns=['idf']).from_dict(
                    dict(idf), orient='index')
        idf.columns = ['idf']
        idf = pd.DataFrame(idf.sort_values(by=['idf'], ascending=True), columns = ['idf'])
        topl = final_topics(idf)
        for j in topl:
            if j not in clust_topics.values():
                clust_topics[i] = j
                break
    
    return list(clust_topics.values())

# Final topics and comments for each topic

In [55]:
#final 5 topics for analyzed product
clust_topics_define(df_clust)

['фианит', 'бриллиант', 'единственный', 'скидка', 'аккуратный']

In [56]:
#first 10 comments for selected topic
num = 1
df_num = list(df_clust[df_clust['cluster'] == num].sort_values(by = ['dist'],ascending = True)['comment'].head(10))
df_num

['Кольцо супер,смотрится как белое золото с бриллиантами, все сверкает на солнце',
 'Очень красивое, изящное и нежное колечко, камушки сверкают',
 'Очень красивое. Нежное. Камушки на солнце играют!!! Спасибо WB и Sokolov',
 'Кольцо прелестное, размер соответствует.\nВыглядит очень достойно, на солнце сверкает 🥰😍',
 'Очень красивое, смотрится дорого (есть с чем сравнить, есть кольцо платина с бриллиантом, очень похожее, только с одим большим камнем). Не знаю на долго ли камень правда... не очень надежное крепление. Камень в меру большой, сверкает) В размер.мне на средний палец подошел. На него и брала)',
 'Колечко прелесть, нежное. На ухоженных длинных пальчиках смотрится изумительно. Оставила, хоть и большеватое. (Да, на белое золото с бриллиантами НЕ ТЯНЕТ, а вот для серебра  ОТЛИЧНО  и явно на бижутерию не смахивает, как некоторые пишут)',
 'Хорошее колечко, получила 20 марта, ни один камушек не выпал. Но носить его надо одно, либо с похожими кольцами. Когда я одеваю обручальное с бр